In [78]:
#%pip install pandas

In [79]:
import pyspark
import matplotlib.pyplot as plt
from pyspark.sql.functions import col, corr
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VarianceThresholdSelector
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import MinMaxScaler, VectorAssembler

In [80]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()
df = spark.read.parquet(r"D:\hacs_spark\4763_str_combined.parquet"
                    ,inferSchema= True)

In [81]:
df.show(5)

+-------+--------------------+---------+-------+-------+-------+-----------+-----------+-----------+-------+-------+-------+-----------+-----------+-----------+
|HotelID|          Hotel Name|     Date|PropSup|PropDem|PropRev|    PropOcc|    PropADR| PropRevPAR|CompSup|CompDem|CompRev|    CompOcc|    CompADR| CompRevPAR|
+-------+--------------------+---------+-------+-------+-------+-----------+-----------+-----------+-------+-------+-------+-----------+-----------+-----------+
|   4763|AC Hotels by Marr...|5/23/2021|    234|    143|  14982|0.611111111|104.7692308|64.02564103|   1311|    677|  71519|0.516399695|105.6410635|54.55301297|
|   4763|AC Hotels by Marr...|5/24/2021|    234|    112|  11946|0.478632479|106.6607143|51.05128205|   1311|    528|  55124|0.402745995|104.4015152|42.04729214|
|   4763|AC Hotels by Marr...|5/25/2021|    234|     99|  10733|0.423076923|108.4141414|45.86752137|   1311|    496|  54884|0.378337147|110.6532258|41.86422578|
|   4763|AC Hotels by Marr...|5/26

In [82]:
df.describe

<bound method DataFrame.describe of DataFrame[HotelID: int, Hotel Name: string, Date: string, PropSup: int, PropDem: int, PropRev: int, PropOcc: double, PropADR: double, PropRevPAR: double, CompSup: int, CompDem: int, CompRev: int, CompOcc: double, CompADR: double, CompRevPAR: double]>

In [83]:

# propDemTY = df.select("PropDemTY").rdd.flatMap(lambda x: x).collect()

# Date = df.select("DateTY").rdd.flatMap(lambda x: x).collect()
# print(type(propDemTY),type(Date))

# # Create a line chart using Matplotlib
# plt.figure(figsize=(8, 6))
# plt.hist(propDemTY, Date,  linestyle='-')
# plt.title("Line Chart Example")
# plt.xlabel("X-axis")
# plt.ylabel("Y-axis")
# plt.grid(True)

# # Show the plot
# plt.show()

 Feature Selection

In [84]:
columns = ['PropSup',
 'PropDem',
 'PropRev',
 'PropOcc',
 'PropADR',
 'PropRevPAR',
 'CompSup',
 'CompDem',
 'CompRev',
 'CompOcc',
 'CompADR',
 'CompRevPAR'
 ]


# Convert numeric columns to a feature vector
assembler = VectorAssembler(inputCols=columns, outputCol="features")
x = assembler.transform(df)

# Calculate the correlation matrix
corr_matrix = Correlation.corr(x, "features").collect()[0][0]
#corr_matrix.show()

In [85]:
# Display the correlation matrix
correlation_matrix = corr_matrix.toArray()

# Print the correlation matrix
for i in range(len(correlation_matrix)):
    print(i)
    for j in range(len(correlation_matrix[i])):
        print(f"Correlation between elements {i} and {j}: {correlation_matrix[i, j]}")

0
Correlation between elements 0 and 0: 1.0
Correlation between elements 0 and 1: nan
Correlation between elements 0 and 2: nan
Correlation between elements 0 and 3: nan
Correlation between elements 0 and 4: nan
Correlation between elements 0 and 5: nan
Correlation between elements 0 and 6: nan
Correlation between elements 0 and 7: nan
Correlation between elements 0 and 8: nan
Correlation between elements 0 and 9: nan
Correlation between elements 0 and 10: nan
Correlation between elements 0 and 11: nan
1
Correlation between elements 1 and 0: nan
Correlation between elements 1 and 1: 1.0
Correlation between elements 1 and 2: 0.8391573898181064
Correlation between elements 1 and 3: 1.0000000000000007
Correlation between elements 1 and 4: 0.696513925086892
Correlation between elements 1 and 5: 0.8391573898190153
Correlation between elements 1 and 6: nan
Correlation between elements 1 and 7: 0.8500615847701802
Correlation between elements 1 and 8: 0.7950765327045598
Correlation between ele

In [86]:
correlation_matrix.shape

(12, 12)

In [87]:
print(str(corr_matrix).replace('nan', 'NaN'))

DenseMatrix([[1.        ,        NaN,        NaN,        NaN,        NaN,
                     NaN,        NaN,        NaN,        NaN,        NaN,
                     NaN,        NaN],
             [       NaN, 1.        , 0.83915739, 1.        , 0.69651393,
              0.83915739,        NaN, 0.85006158, 0.79507653, 0.85006158,
              0.66548285, 0.79507653],
             [       NaN, 0.83915739, 1.        , 0.83915739, 0.96752775,
              1.        ,        NaN, 0.74334807, 0.93190002, 0.74334807,
              0.90225791, 0.93190002],
             [       NaN, 1.        , 0.83915739, 1.        , 0.69651393,
              0.83915739,        NaN, 0.85006158, 0.79507653, 0.85006158,
              0.66548285, 0.79507653],
             [       NaN, 0.69651393, 0.96752775, 0.69651393, 1.        ,
              0.96752775,        NaN, 0.64171226, 0.90810034, 0.64171226,
              0.93836359, 0.90810034],
             [       NaN, 0.83915739, 1.        , 0.83915739, 0.9

Variance Threshold Selector

In [88]:
features1=x.select("features")

In [89]:
features1.show()

+--------------------+
|            features|
+--------------------+
|[234.0,143.0,1498...|
|[234.0,112.0,1194...|
|[234.0,99.0,10733...|
|[234.0,112.0,1186...|
|[234.0,112.0,1110...|
|[234.0,172.0,1972...|
|[234.0,215.0,2650...|
|[234.0,167.0,1906...|
|[234.0,101.0,1010...|
|[234.0,86.0,8876....|
|[234.0,81.0,8420....|
|[234.0,102.0,1058...|
|[234.0,135.0,1496...|
|[234.0,163.0,1888...|
|[234.0,107.0,1123...|
|[234.0,89.0,9848....|
|[234.0,113.0,1238...|
|[234.0,115.0,1325...|
|[234.0,139.0,1472...|
|[234.0,181.0,2006...|
+--------------------+
only showing top 20 rows



In [90]:
selector = VarianceThresholdSelector(varianceThreshold=0.5, outputCol="selectedFeatures")

result = selector.fit(features1)#.transform(features1)

# print("Features selected by VarianceThresholdSelector:")
# result.show()

result.getFeaturesCol()

result.setFeaturesCol("features")
#VarianceThresholdSelectorModel...
result.transform(features1).head().selectedFeatures

result.selectedFeatures

[1, 2, 4, 5, 7, 8, 10, 11]

Modelling

In [91]:
df.show()

+-------+--------------------+---------+-------+-------+-------+-----------+-----------+-----------+-------+-------+-------+-----------+-----------+-----------+
|HotelID|          Hotel Name|     Date|PropSup|PropDem|PropRev|    PropOcc|    PropADR| PropRevPAR|CompSup|CompDem|CompRev|    CompOcc|    CompADR| CompRevPAR|
+-------+--------------------+---------+-------+-------+-------+-----------+-----------+-----------+-------+-------+-------+-----------+-----------+-----------+
|   4763|AC Hotels by Marr...|5/23/2021|    234|    143|  14982|0.611111111|104.7692308|64.02564103|   1311|    677|  71519|0.516399695|105.6410635|54.55301297|
|   4763|AC Hotels by Marr...|5/24/2021|    234|    112|  11946|0.478632479|106.6607143|51.05128205|   1311|    528|  55124|0.402745995|104.4015152|42.04729214|
|   4763|AC Hotels by Marr...|5/25/2021|    234|     99|  10733|0.423076923|108.4141414|45.86752137|   1311|    496|  54884|0.378337147|110.6532258|41.86422578|
|   4763|AC Hotels by Marr...|5/26

In [114]:
x=df.select("PropOcc",'PropADR',
 'PropRevPAR',
 'CompOcc',
 'CompADR',
 'CompRevPAR')
x.show()

+-----------+-----------+-----------+-----------+-----------+-----------+
|    PropOcc|    PropADR| PropRevPAR|    CompOcc|    CompADR| CompRevPAR|
+-----------+-----------+-----------+-----------+-----------+-----------+
|0.611111111|104.7692308|64.02564103|0.516399695|105.6410635|54.55301297|
|0.478632479|106.6607143|51.05128205|0.402745995|104.4015152|42.04729214|
|0.423076923|108.4141414|45.86752137|0.378337147|110.6532258|41.86422578|
|0.478632479|105.9642857|50.71794872|0.366132723|111.4895833|40.81998474|
|0.478632479|99.10714286|47.43589744|0.435545385|109.0507881|47.49656751|
|0.735042735|114.7034884|84.31196581|0.675057208| 122.919774|82.97787948|
|0.918803419|123.2697674|113.2606838|0.868802441| 135.570676|117.7841342|
|0.713675214|114.1377246|81.45726496|0.718535469|112.4617834|80.80778032|
|0.431623932|100.0594059|43.18803419|0.382913806|108.1374502|41.40732265|
|0.367521368|103.2093023|37.93162393|0.315026697| 113.188862|35.65751335|
|0.346153846|103.9506173|35.98290598|0

In [121]:
from pyspark.ml import Pipeline

# Select the relevant columns for your features
feature_columns = ["PropOcc",'PropADR',
 'PropRevPAR',
 'CompOcc',
 'CompADR',
 'CompRevPAR']

scaled_df = x
# Step 1: Vectorize the data
for feature in feature_columns:
    assembler = VectorAssembler(inputCols=[feature], outputCol=f"vec{feature}")
    vectorized_df = assembler.transform(scaled_df)

#vectorized_df.show(5)
    
#Step 2: Apply Min-Max Scaling
    scaler = MinMaxScaler(inputCol=f"vec{feature}", outputCol=f"scaled{feature}")
    scaler_model = scaler.fit(vectorized_df)
    scaled_df = scaler_model.transform(vectorized_df)
    
scaled_df.show(5)
# # Rename the target column to "label" if it's not already named that way
# scaled_df = scaled_df.withColumnRenamed("PropOcc", "label")
# scaled_df.show()
# # Create a pipeline to execute the transformations
# pipeline = Pipeline(stages=[assembler])

# # Fit and transform the data
# model_data = pipeline.fit(scaled_df).transform(scaled_df)
# model_data.show()

# # Now you have the "features" column containing the feature vectors and the "label" column for your target variable

# # You can split the data into training and testing sets
# train_data, test_data = model_data.randomSplit([0.8, 0.2], seed=42)







+-----------+-----------+-----------+-----------+-----------+-----------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+
|    PropOcc|    PropADR| PropRevPAR|    CompOcc|    CompADR| CompRevPAR|   vecPropOcc|       scaledPropOcc|   vecPropADR|       scaledPropADR|vecPropRevPAR|    scaledPropRevPAR|   vecCompOcc|       scaledCompOcc|   vecCompADR|       scaledCompADR|vecCompRevPAR|    scaledCompRevPAR|
+-----------+-----------+-----------+-----------+-----------+-----------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+
|0.611111111|104.7692308|64.02564103|0.516399695|105.6410635|54.55301297|[0.611111111]|[0.5159574464178928]|[104.7692308]|[0.04291756807483...|[64.0

In [94]:
x=scaled_df.select("scaledPropOcc","scaledPropADR","scaledPropRevPAR","scaledCompOcc","scaledCompADR","scaledCompRevPAR")
x.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|       scaledPropOcc|       scaledPropADR|    scaledPropRevPAR|       scaledCompOcc|       scaledCompADR|    scaledCompRevPAR|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[0.5159574464178928]|[0.04291756807483...|[0.09928789633459...|[0.42100192674942...|[0.04429639741887...|[0.09951218362586...|
|[0.35106382990640...|[0.04839727377868...|[0.06763421016662...|[0.27745664655927...|[0.03962379586337...|[0.06087571285316...|
|[0.2819148931469556]|[0.05347702439831...|[0.05498733227558...|[0.24662813041524...|[0.06319024557258...|[0.06031012865663...|
|[0.35106382990640...|[0.04637969140392...|[0.0668209731786987]| [0.231213872343229]|[0.0663429789960871]|[0.05708394210009...|
|[0.35106382990640...|[0.02651426580942...|[0.05881371660714...|[0.31888246575931...|[0.0571496960517562

In [110]:
from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vectors
from pyspark.sql.types import DoubleType

# Define a UDF to extract elements from a vector
extract_elements = udf(lambda vec: [float(e) for e in vec], DoubleType())
